In [ ]:
import os
import urllib.request
from urllib.parse import urljoin
from bs4 import BeautifulSoup

import numpy as np
from astropy.io import fits
from astropy.table import Table as t, vstack

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.ticker import MaxNLocator, AutoMinorLocator
matplotlib.rcParams['figure.dpi'] = 360
matplotlib.rcParams['text.usetex'] = True
os.environ['PATH'] = '/Library/TeX/texbin:' + os.environ['PATH']
cmap = sns.color_palette('mako', 10)

In [ ]:
BASE_URL = 'https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/'

OUT_DIR = './data/edr/vac/edr/lss/v2.0/LSScats/clustering/'
os.makedirs(OUT_DIR, exist_ok=True)

In [ ]:
def list_links(url):
    resp = urllib.request.urlopen(url)
    soup = BeautifulSoup(resp, 'html.parser')
    return [a['href'] for a in soup.find_all('a', href=True)
            if not a['href'].startswith('?') and a['href'] != '../']

In [ ]:
files = list_links(BASE_URL)
elg = [fn for fn in files if ('ELG_N_' in fn) or ('ELG_S_' in fn)]
len(elg)

In [ ]:
for file in elg:
    url = urljoin(BASE_URL, file)
    out_file = os.path.join(OUT_DIR, file)
    if not os.path.exists(out_file):
        try:
            urllib.request.urlretrieve(url, out_file)
        except Exception as e:
            print(f'{url}: {e}')

In [ ]:
files_elg = [os.path.join(OUT_DIR, fn) for fn in elg]

data = [fn for fn in files_elg if '.dat' in fn]
ran_n = [fn for fn in files_elg if ('ELG_N_' in fn) and ('ran' in fn)]
ran_s = [fn for fn in files_elg if ('ELG_S_' in fn) and ('ran' in fn)]

f_n, f_s = fits.open(data[0]), fits.open(data[1])
data

### North

In [ ]:
table_n = t(f_n['LSS'].data)
table_n[:5]

In [ ]:
fig, ax = plt.subplots()
ax.grid(linewidth=0.2, zorder=0)

for i, rn in enumerate(np.unique(table_n['ROSETTE_NUMBER'])):
    data_r = table_n[table_n['ROSETTE_NUMBER'] == rn]
    ra, dec = data_r['RA'], data_r['DEC']
    ax.scatter(ra, dec, s=0.1, color=cmap[i], label=f'Rosette {rn}')

ax.legend(loc='lower left', markerscale=12)
ax.set_xlabel(r'RA [deg]'); ax.set_ylabel(r'DEC [deg]')

plt.tight_layout()
plt.show()

In [ ]:
for i, rn in enumerate(np.unique(table_n['ROSETTE_NUMBER'])):
    fig, ax = plt.subplots()
    data_r = table_n[table_n['ROSETTE_NUMBER'] == rn]
    ra, dec = data_r['RA'], data_r['DEC']

    ax.grid(linewidth=0.2, zorder=0)
    ax.scatter(ra, dec, s=2, color=cmap[i])
    ax.set_xlabel('RA [deg]')
    ax.set_ylabel('DEC [deg]')
    ax.set_title(f'Rosette {rn}', y=1.02)

    ax.set_box_aspect(1)
    plt.tight_layout()
    plt.show()

In [ ]:
fig, axes = plt.subplots(2, 5, sharey=True, figsize=(12, 5))

for i, rn in enumerate(np.unique(table_n['ROSETTE_NUMBER'])):
    data_r = table_n[table_n['ROSETTE_NUMBER'] == rn]
    ax = axes[i//5, i%5]
    ax.grid(linewidth=0.2, zorder=0)
    ax.hist(data_r['Z'], bins=30, color=[cmap[i]], edgecolor='black', alpha=0.8,
            zorder=10, label=f'Rosette {rn}')
    ax.legend(loc='upper right', fontsize=8)

    if (i//5==1):
        ax.set_xlabel('$Z$')
    ax.xaxis.set_major_locator(MaxNLocator(nbins=6, prune='both'))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=5, prune='both'))

    ax.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))

    ax.tick_params(which='major', length=6, width=1)
    ax.tick_params(which='minor', length=3, width=0.5)

axes[0,0].set_ylabel('Counts'); axes[1,0].set_ylabel('Counts')
plt.tight_layout()
plt.show()

In [ ]:
cuts_n = [(0.93,0.96), (0.96,1.0), (0.83,0.9), (0.86,0.9), (0.96,1.0),
        (0.86,0.9), (0.83,0.86), (0.96,1.0), (0.86,0.9), (0.93,0.96)]

In [ ]:
for i, rn in enumerate(np.unique(table_n['ROSETTE_NUMBER'])):
    fig, ax = plt.subplots()
    mask = (table_n['ROSETTE_NUMBER'] == rn) & (table_n['Z'] > cuts_n[i][0]) & (table_n['Z'] < cuts_n[i][1])
    data_r = table_n[mask]
    ra, dec = data_r['RA'], data_r['DEC']

    ax.grid(linewidth=0.2, zorder=0)
    ax.scatter(ra, dec, s=2, color=cmap[i])

    ax.set_xlabel('RA [deg]')
    ax.set_ylabel('DEC [deg]')
    ax.set_title(f'Rosette {rn}', y=1.02)

    ax.set_box_aspect(1)
    plt.tight_layout()
    plt.show()

In [ ]:
tables_n = []
for i in range(len(ran_n)):
    tab = t.read(f'./data/edr/vac/edr/lss/v2.0/LSScats/clustering/ELG_N_{i}_clustering.ran.fits', hdu='LSS')
    tables_n.append(tab)

combined_n = vstack(tables_n)

In [ ]:
for i, rn in enumerate(np.unique(table_n['ROSETTE_NUMBER'])):
    fig, ax = plt.subplots()
    mask = (table_n['ROSETTE_NUMBER'] == rn) & (table_n['Z'] > cuts_n[i][0]) & (table_n['Z'] < cuts_n[i][1])
    data_d = table_n[mask]

    mask = (combined_n['ROSETTE_NUMBER'] == rn) & (combined_n['Z'] > cuts_n[i][0]) & (combined_n['Z'] < cuts_n[i][1])
    data_r = combined_n[mask]

    ax.grid(linewidth=0.2, zorder=0)
    ax.scatter(data_r['RA'], data_r['DEC'], s=0.1, color='b', label='Random')
    ax.scatter(data_d['RA'], data_d['DEC'], s=0.4, color='r', label='Data')

    ax.set_xlabel('RA [deg]')
    ax.set_ylabel('DEC [deg]')
    ax.set_title(f'Rosette {rn}', y=1.02)

    ax.legend(loc='upper right')
    ax.set_box_aspect(1)
    plt.tight_layout()
    plt.show()

### South

In [ ]:
table_s = t(f_s['LSS'].data)

In [ ]:
fig, ax = plt.subplots()
ax.grid(linewidth=0.2, zorder=0)

for i, rn in enumerate(np.unique(table_s['ROSETTE_NUMBER'])):
    data_r = table_s[table_s['ROSETTE_NUMBER'] == rn]
    ra, dec = data_r['RA'], data_r['DEC']
    ax.scatter(ra, dec, s=0.1, color=cmap[i], label=f'Rosette {rn}')

ax.legend(loc='upper left', markerscale=12)
ax.set_xlabel(r'RA [deg]'); ax.set_ylabel(r'DEC [deg]')

plt.tight_layout()
plt.show()

In [ ]:
for i, rn in enumerate(np.unique(table_s['ROSETTE_NUMBER'])):
    fig, ax = plt.subplots()
    data_r = table_s[table_s['ROSETTE_NUMBER'] == rn]
    ra, dec = data_r['RA'], data_r['DEC']

    ax.grid(linewidth=0.2, zorder=0)
    ax.scatter(ra, dec, s=2, color=cmap[i])
    ax.set_xlabel('RA [deg]')
    ax.set_ylabel('DEC [deg]')
    ax.set_title(f'Rosette {rn}', y=1.02)

    ax.set_box_aspect(1)
    plt.tight_layout()
    plt.show()

In [ ]:
fig, axes = plt.subplots(2, 5, sharey=True, figsize=(12, 5))

for i, rn in enumerate(np.unique(table_s['ROSETTE_NUMBER'])):
    data_r = table_s[table_s['ROSETTE_NUMBER'] == rn]
    ax = axes[i//5, i%5]
    ax.grid(linewidth=0.2, zorder=0)
    ax.hist(data_r['Z'], bins=30, color=[cmap[i]], edgecolor='black', alpha=0.8,
            zorder=10, label=f'Rosette {rn}')
    ax.legend(loc='upper right', fontsize=8)

    if (i//5==1):
        ax.set_xlabel('$Z$')
    ax.xaxis.set_major_locator(MaxNLocator(nbins=6, prune='both'))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=5, prune='both'))

    ax.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))

    ax.tick_params(which='major', length=6, width=1)
    ax.tick_params(which='minor', length=3, width=0.5)

axes[0,0].set_ylabel('Counts'); axes[1,0].set_ylabel('Counts')
plt.tight_layout()
plt.show()

In [ ]:
cuts_s = [(0.93,0.96), (0.93,0.96), (0.93,0.96), (0.96,1.0), (0.9,0.93),
        (0.83,0.86), (0.86,0.9), (0.96,1.0), (0.96,1.0), (0.83,0.86)]

In [ ]:
for i, rn in enumerate(np.unique(table_s['ROSETTE_NUMBER'])):
    fig, ax = plt.subplots()
    mask = (table_s['ROSETTE_NUMBER'] == rn) & (table_s['Z'] > cuts_s[i][0]) & (table_s['Z'] < cuts_s[i][1])
    data_r = table_s[mask]
    ra, dec = data_r['RA'], data_r['DEC']

    ax.grid(linewidth=0.2, zorder=0)
    ax.scatter(ra, dec, s=2, color=cmap[i])

    ax.set_xlabel('RA [deg]')
    ax.set_ylabel('DEC [deg]')
    ax.set_title(f'Rosette {rn}', y=1.02)

    ax.set_box_aspect(1)
    plt.tight_layout()
    plt.show()

In [ ]:
tables_s = []
for i in range(len(ran_n)):
    tab = t.read(f'./data/edr/vac/edr/lss/v2.0/LSScats/clustering/ELG_S_{i}_clustering.ran.fits', hdu='LSS')
    tables_s.append(tab)

combined_s = vstack(tables_s)

In [ ]:
for i, rn in enumerate(np.unique(table_s['ROSETTE_NUMBER'])):
    fig, ax = plt.subplots()
    mask = (table_s['ROSETTE_NUMBER'] == rn) & (table_s['Z'] > cuts_s[i][0]) & (table_s['Z'] < cuts_s[i][1])
    data_d = table_s[mask]

    mask = (combined_s['ROSETTE_NUMBER'] == rn) & (combined_s['Z'] > cuts_s[i][0]) & (combined_s['Z'] < cuts_s[i][1])
    data_r = combined_s[mask]

    ax.grid(linewidth=0.2, zorder=0)
    ax.scatter(data_r['RA'], data_r['DEC'], s=0.1, color='b', label='Random')
    ax.scatter(data_d['RA'], data_d['DEC'], s=0.4, color='r', label='Data')

    ax.set_xlabel('RA [deg]')
    ax.set_ylabel('DEC [deg]')
    ax.set_title(f'Rosette {rn}', y=1.02)

    ax.legend(loc='upper right')
    ax.set_box_aspect(1)
    plt.tight_layout()
    plt.show()